In [1]:
from bs4 import BeautifulSoup
import requests
import unicodecsv

In [2]:
base1_url='https://urbania.pe'
base_url='https://urbania.pe/buscar/venta-de-casas-en-lima'
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
 

In [3]:
def get_items(response):
    soup = BeautifulSoup(response.content,'lxml')
    items = soup.find_all('section', {'class': 'b-card js-results-of-search'})[0]

    '''    
    ar_tags = soup.find_all('div', {'class': 'b-card-wrap js-card-list'})[0]
    
    ur_tags = ar_tags.find_all('a')
    a_tags=[]
    for a_tag in ar_tags:
        a_tags.extend(ar_tags.find_all('a'))
    a_tags = [ a_tag for a_tag in a_tags if 'href' in a_tag.attrs ]
    links  = [ a_tag.get('href')  for a_tag in a_tags ]
    print (base1_url+links[0])
'''
    item_list = []

    for li in items.find_all('article'):
        item = {}
        title = li.find_all('h2')[0].text.strip()
        price = li.find_all('span',{'class': 'b-card-currency-first'})[0].text.strip()
        link = base1_url+li.find_all('a')[0].get('href')
        #a_tags = []
        #for a_tag in l_tags:
        #    a_tags.extend(l_tags.find_all('a'))
        #a_tags = [a_tag for a_tag in a_tags if 'href' in a_tag.attrs]
        #links  = [ a_tag.get('href')  for a_tag in a_tags ]
        #link = links[0].text.strip()
        dire = li.find_all('h4')[0].text.strip()
        distr = li.find_all('h3')[0].text.strip()
        content = requests.get(link, headers=headers).content
        soup2 = BeautifulSoup(content,'lxml')
        charac = soup2.find_all('div', {'class': 'b-leading-data-property u-flex-wrap'})
        for lo in charac:
            #if lo.find_all('div', {'class': 'e-leading-data-service'})
            a = len(lo.find_all('p',{'class': 'e-leading-data-service'}))
            #a sera el numero maximo de 
            for i in range (0,a):
                #Dado que el formato del xml de caracteristicas estan de manera consecutiva sin diferenciar
                #hacemos un if para que cada posicion par sean las cabeceras y las impares sean los datos
                #esto lo hacemos pues hay casas que no tienen todos los datos, es decir que son variables
                if i%2==0:
                    cabeceras = lo.find_all('p',{'class': 'e-leading-data-service'})[i].text.strip()
                    #print(cabeceras)
                else:
                    datos = lo.find_all('p',{'class': 'e-leading-data-service'})[i].text.strip()
                #m = "dato"+str(i+1)+"_head"
                #m1 = "dato"+str(i+1)
                #n = lo.find_all('p',{'class': 'e-leading-data-service'})[i].text.strip()
                #n1 = lo.find_all('p',{'class': 'e-leading-data-service'})[i+1].text.strip()
                    #print (cabeceras,datos)
                    item[cabeceras]=datos
                    item['Titulo'] = title
                    item['Precio'] = price
                    item['Direccion'] = dire
                    item['Distrito'] = distr
                    item['Link'] = link
        #item['Tipo'] = tipo
        #item['Antiguedad']=antiguedad
        #item['Características'] = charac.
            item_list.append(item)
            
    return item_list

In [ ]:
#Lista de diccionarios
merged_list = []
n = int(input('Ingresa cuantas paginas necesitas scrapear (máximo de 528)'))
for i in range(n):
    print ('Scrapeando Urbania PAGINA: '+str(i))
    r = requests.get(base_url+'?page='+str(i+1))
    merged_list.extend(get_items(r))
    

Ingresa cuantas paginas necesitas scrapear (máximo de 528)528
Scrapeando Urbania PAGINA: 0
Scrapeando Urbania PAGINA: 1
Scrapeando Urbania PAGINA: 2
Scrapeando Urbania PAGINA: 3
Scrapeando Urbania PAGINA: 4
Scrapeando Urbania PAGINA: 5
Scrapeando Urbania PAGINA: 6
Scrapeando Urbania PAGINA: 7
Scrapeando Urbania PAGINA: 8
Scrapeando Urbania PAGINA: 9
Scrapeando Urbania PAGINA: 10
Scrapeando Urbania PAGINA: 11
Scrapeando Urbania PAGINA: 12
Scrapeando Urbania PAGINA: 13
Scrapeando Urbania PAGINA: 14
Scrapeando Urbania PAGINA: 15
Scrapeando Urbania PAGINA: 16
Scrapeando Urbania PAGINA: 17
Scrapeando Urbania PAGINA: 18
Scrapeando Urbania PAGINA: 19
Scrapeando Urbania PAGINA: 20
Scrapeando Urbania PAGINA: 21
Scrapeando Urbania PAGINA: 22
Scrapeando Urbania PAGINA: 23
Scrapeando Urbania PAGINA: 24
Scrapeando Urbania PAGINA: 25
Scrapeando Urbania PAGINA: 26
Scrapeando Urbania PAGINA: 27
Scrapeando Urbania PAGINA: 28
Scrapeando Urbania PAGINA: 29
Scrapeando Urbania PAGINA: 30
Scrapeando Urbania

In [5]:
with open('urbania.csv','wb') as f:
    dict_writer = unicodecsv.DictWriter(f, ['Titulo','Precio','Direccion','Distrito','Link', 'Tipo', 'Antiguedad','Dormitorios','Baños', 'Área Total', 'Área Construida', 'Area de Terreno', 'Urbanización', 'Ubicación', 'Referencia', 'Mascotas', 'Cocheras', 'Balneario', 'Fila'],encoding='utf-8')
    dict_writer.writeheader()
    dict_writer.writerows(merged_list)